In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pickle


# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [2]:
ak = pd.read_csv('../datasets/alaska_single_engine_clean.csv', low_memory=False)

In [3]:
ak.shape

(5590, 35)

In [4]:
ak = ak.drop(['ntsb_no', 'mkey', 'n', 'probable_cause', 'event_time', 'fatal_injury_count', 'serious_injury_count', 'minor_injury_count', 'aircraft_damage', 'event_type', 'report_type', 'has_safety_rec', 'highest_injury_level', 'is_accident', 'has_aircraft_damage'], axis=1)

In [5]:
categorical_cols = ak.select_dtypes(exclude=['number']).columns.tolist()

In [6]:
categorical_cols

['city',
 'make',
 'model',
 'aircraft_category',
 'airport_id',
 'airport_name',
 'scheduled',
 'purpose_of_flight',
 'far',
 'weather_condition',
 'operator',
 'event_season']

In [7]:
ak.columns

Index(['city', 'latitude', 'longitude', 'make', 'model', 'aircraft_category',
       'airport_id', 'airport_name', 'amateur_built', 'scheduled',
       'purpose_of_flight', 'far', 'weather_condition', 'operator',
       'event_year', 'event_month', 'event_day', 'event_hour', 'event_season',
       'has_injury'],
      dtype='object')

In [8]:
ak_dummies = pd.get_dummies(ak, columns = categorical_cols, drop_first=True)

In [9]:
X = ak_dummies.drop('has_injury', axis=1)
y = ak_dummies['has_injury']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.075, random_state=21)

### Baseline

In [11]:
X.columns.value_counts().sum()

6587

In [12]:
1 - y.mean()

0.716994633273703

### Benchmark Logistic Regression Model

In [13]:
pipe_log_reg_bench = Pipeline([
    ('sc', StandardScaler()),
    ('log_reg_bench', LogisticRegression(max_iter=1000))
])

In [14]:
pipe_log_reg_bench.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('log_reg_bench', LogisticRegression(max_iter=1000))])

In [15]:
pipe_log_reg_bench.score(X_train, y_train)

0.9665377176015474

In [16]:
pipe_log_reg_bench.score(X_test, y_test)

0.6833333333333333

### Tuned Logistic Regression Model

In [17]:
pipe_log_reg_tuned = Pipeline([
    ('sc', StandardScaler()),
    ('log_reg_tuned', LogisticRegression())
])

In [18]:
pipe_log_reg_tuned.get_params()

{'memory': None,
 'steps': [('sc', StandardScaler()), ('log_reg_tuned', LogisticRegression())],
 'verbose': False,
 'sc': StandardScaler(),
 'log_reg_tuned': LogisticRegression(),
 'sc__copy': True,
 'sc__with_mean': True,
 'sc__with_std': True,
 'log_reg_tuned__C': 1.0,
 'log_reg_tuned__class_weight': None,
 'log_reg_tuned__dual': False,
 'log_reg_tuned__fit_intercept': True,
 'log_reg_tuned__intercept_scaling': 1,
 'log_reg_tuned__l1_ratio': None,
 'log_reg_tuned__max_iter': 100,
 'log_reg_tuned__multi_class': 'auto',
 'log_reg_tuned__n_jobs': None,
 'log_reg_tuned__penalty': 'l2',
 'log_reg_tuned__random_state': None,
 'log_reg_tuned__solver': 'lbfgs',
 'log_reg_tuned__tol': 0.0001,
 'log_reg_tuned__verbose': 0,
 'log_reg_tuned__warm_start': False}

In [19]:
pipe_log_reg_tuned_params = {
    'log_reg_tuned__max_iter': [100, 1_000],
    'log_reg_tuned__C': np.linspace(0.00001, .01, 5),
    'log_reg_tuned__penalty': ['l1', 'l2']
}

In [20]:
gs= GridSearchCV(pipe_log_reg_tuned,
                                param_grid = pipe_log_reg_tuned_params,
                                cv = 5,
                               n_jobs=6)

In [21]:
%%time
gs.fit(X_train, y_train)

/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 420,

CPU times: user 4.57 s, sys: 1.04 s, total: 5.6 s
Wall time: 1min 14s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('sc', StandardScaler()),
                                       ('log_reg_tuned',
                                        LogisticRegression())]),
             n_jobs=6,
             param_grid={'log_reg_tuned__C': array([1.0000e-05, 2.5075e-03, 5.0050e-03, 7.5025e-03, 1.0000e-02]),
                         'log_reg_tuned__max_iter': [100, 1000],
                         'log_reg_tuned__penalty': ['l1', 'l2']})

In [22]:
gs.best_params_

{'log_reg_tuned__C': 1e-05,
 'log_reg_tuned__max_iter': 100,
 'log_reg_tuned__penalty': 'l2'}

In [23]:
gs.score(X_train, y_train)

0.7156673114119922

In [24]:
gs.score(X_test, y_test)

0.7333333333333333

In [25]:
y_pred = gs.predict(X_test)

In [26]:
confusion_matrix(y_test, y_pred)

array([[308,   0],
       [112,   0]])

In [27]:
print(classification_report(y_test, y_pred, digits = 5))

              precision    recall  f1-score   support

           0    0.73333   1.00000   0.84615       308
           1    0.00000   0.00000   0.00000       112

    accuracy                        0.73333       420
   macro avg    0.36667   0.50000   0.42308       420
weighted avg    0.53778   0.73333   0.62051       420



/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nolan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Coefficients

In [28]:
# coefficients = np.exp(gs.best_estimator_.named_steps['log_reg_tuned'].coef_[0]*.1)

In [29]:
coefficients = gs.best_estimator_.named_steps['log_reg_tuned'].coef_[0]

In [30]:
feature_names = list(X_train.columns)

In [31]:
coefficients_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefficients})

In [32]:
pd.set_option('display.max_rows', None)
coefficients_df.head(25)

,feature,coefficient
0,latitude,-0.000793
1,longitude,0.000741
2,amateur_built,0.001154
3,event_year,-0.001222
4,event_month,0.000229
5,event_day,0.000108
6,event_hour,-0.000878
7,city_100MI S.KNG SLM,-0.000205
8,city_11NM EAST OF SI,0.000501
9,city_18NM ESE KETCHI,-0.000197


In [33]:
coefficients_df.sort_values(by='coefficient', ascending=False).head(30)

,feature,coefficient
4186,weather_condition_Unknown,0.002491
3782,airport_name_No Airport,0.002182
4183,far_135,0.002125
4175,purpose_of_flight_UNK,0.001917
2862,airport_id_No Airport,0.001731
2,amateur_built,0.001154
483,city_KETCHIKAN,0.001098
2517,aircraft_category_HELI,0.001092
536,city_Ketchikan,0.001077
470,city_KASILOF,0.001042


In [34]:
coefficients_df.sort_values(by='coefficient', ascending=False).tail(30)

,feature,coefficient
383,city_Gulkana,-0.000629
2166,model_M-5-210C,-0.000630
2702,airport_id_ANC,-0.000648
2870,airport_id_ORT,-0.000651
127,city_Beluga,-0.000654
2848,airport_id_MRI,-0.000662
2329,model_PA18,-0.000663
4166,purpose_of_flight_INST,-0.000673
2951,airport_id_PATK,-0.000685
316,city_FAIRBANKS,-0.000701


In [35]:
coefficients_df.to_csv('../datasets/nolan_log_reg_coef.csv', index=False)

### Pickle

In [36]:
with open ('../pickles/nolan_log_reg.pkl', 'wb') as f:
    pickle.dump('gs.pkl', f)